In [4]:
import requests
from bs4 import BeautifulSoup

class RequestCustom:
    def __init__(self, url, site_name):
        self.url = url
        self.site_name = site_name

    def get_url(self):
        return self.url

    def get_request(self):
        url=self.get_url()     
        return requests.get(url, allow_redirects=False)

    def get_status(self):
        request = self.get_request()
        status = request.status_code
        return status

    def get_soup(self):
        request=self.get_request()
        status = self.get_status()
        if status == 200:
            soup = BeautifulSoup(request.text, 'html.parser')
            return soup
        else:
            print("Error: ", status)
            return None

In [5]:
import csv

class CsvCustom:

    def __init__(self, site_name):
        self.site_name = site_name

    def create_csv(self):
        file = open(f'{self.site_name}.csv', 'w',newline='',encoding='utf-8-sig')
        writer = csv.writer(file)
        headers = ["title", "link", "description", "price"]
        writer.writerow(headers)

    def write_csv(self,**kwargs): 
        title = kwargs['title']
        link = kwargs['link']
        description = kwargs['description']
        price = kwargs['price']

        file = open(f'{self.site_name}.csv', 'a', newline='', encoding='utf-8-sig')
        writer = csv.writer(file)
        contents = ([title,link, description, price])
        writer.writerow(contents)
        return file

    def close_csv(self,file):
        file.close()

In [6]:
number=1

site_quel_frigo = RequestCustom(url=f"https://www.quelfrigo.com/all-products/page/{number}.html", site_name="quelfrigo")
status = site_quel_frigo.get_status()

csv_test=CsvCustom("quelfrigo")
csv_test.create_csv()

while status == 200:
    #listes_prods
    soup = site_quel_frigo.get_soup()
    listes_prods=soup.find_all("div", attrs={"class":"listes_prods"})


    for prods in listes_prods:
        title = prods.find("span", attrs={"itemprop":"model"}).get_text()
        link = prods.find("a",attrs={"itemprop":"url"})['href']
        description = prods.find("div", attrs={"class":"marqueDescription"}).find("p").get_text()
        price = prods.find("span", attrs={"class":"unite2"}).get_text()

        description = description.strip()
        price = ''.join(price.split())
        
        file=csv_test.write_csv(title=title, link=link, description=description, price=price)
    number=number+1
    site_quel_frigo = RequestCustom(url=f"https://www.quelfrigo.com/all-products/page/{number}.html", site_name="quelfrigo")
    status = site_quel_frigo.get_status()
    
    csv_test.close_csv(file)  

if status == 302:
    print(f"The page {number} doesn't exist")

KeyboardInterrupt: 